In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = (
    SparkSession
        .builder
        .appName("spark_prj_1")
        .config("spark.jars", "/home/long.vk@citigo.id/Downloads/mysql-connector-j-8.0.33.jar")
        .getOrCreate()
)

24/08/05 16:40:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
url = "jdbc:mysql://localhost:3306"

properties = {
    "user": "admin",
    "password": "password",
    "driver": "com.mysql.jdbc.Driver"
}

tables = ["users", "accounts"]

In [4]:
df_users = (
    spark
    .read
    .format("jdbc")
    .option("url", f"{url}/spark_prj_1")
    .option("driver", properties["driver"])
    .option("dbtable", tables[0])
    .option("user", properties["user"])
    .option("password", properties["password"])
    .load()
)

Loading class `com.mysql.jdbc.Driver'. This is deprecated. The new driver class is `com.mysql.cj.jdbc.Driver'. The driver is automatically registered via the SPI and manual loading of the driver class is generally unnecessary.


In [5]:
df_users.printSchema

<bound method DataFrame.printSchema of DataFrame[id: int, first_name: string, last_name: string, date_of_birth: date, age: int, address: string]>

In [6]:
df_users = df_users.withColumnRenamed("id", "user_id")

In [7]:
df_users.show(5)

+-------+-----------+---------+-------------+---+--------------------+
|user_id| first_name|last_name|date_of_birth|age|             address|
+-------+-----------+---------+-------------+---+--------------------+
|      1|       Paul|   Wright|   1966-06-11| 58|0083 Alex Field\n...|
|      2|       Ruth|    James|   1979-09-11| 44|536 Johnson Lodge...|
|      3|   Fernando|   Pineda|   1954-04-02| 70|50542 Roberts Inl...|
|      4|Christopher|     Webb|   1967-02-20| 57|819 Warren Terrac...|
|      5|     Joseph|    Scott|   1959-09-09| 64|974 David Estates...|
+-------+-----------+---------+-------------+---+--------------------+
only showing top 5 rows



In [8]:
(df_users
    .write
    .format("jdbc")
    .option("url", f"{url}/spark_prj_1_sample_dw")
    .option("driver", properties["driver"])
    .option("dbtable", "user_dim")
    .option("user", properties["user"])
    .option("password", properties["password"])
    .mode("append")
    .save()
)

In [9]:
df_accounts = (
    spark
    .read
    .format("jdbc")
    .option("url", f"{url}/spark_prj_1")
    .option("driver", properties["driver"])
    .option("dbtable", tables[1])
    .option("user", properties["user"])
    .option("password", properties["password"])
    .load()
)

In [10]:
df_accounts.show(5)

+---+-------------+----------+--------------------+-----------+-------+
| id|     username|  password|               email|createdDate|user_id|
+---+-------------+----------+--------------------+-----------+-------+
|  1|  lindarogers|8AfMY#Jb_e|rachel63@example.com| 2023-06-20|      1|
|  2|     joseph12|oGnA^KzC@5|jamesjesse@exampl...| 2022-02-01|      1|
|  3|   fhernandez|E$537_BmX+|fergusontonya@exa...| 2018-08-01|      2|
|  4|  paulpearson|$^k0NBg&70|   max00@example.org| 2018-11-09|      2|
|  5|mcleanmatthew|xhsKQ)n4^0|johnrosales@examp...| 2024-03-14|      2|
+---+-------------+----------+--------------------+-----------+-------+
only showing top 5 rows



In [15]:
df_accounts = (df_accounts
     .withColumnRenamed("id", "acc_id")
     .drop("password")
     .drop("createdDate")
     .drop("user_id")
)

In [16]:
df_accounts.show(5)

+------+-------------+--------------------+
|acc_id|     username|               email|
+------+-------------+--------------------+
|     1|  lindarogers|rachel63@example.com|
|     2|     joseph12|jamesjesse@exampl...|
|     3|   fhernandez|fergusontonya@exa...|
|     4|  paulpearson|   max00@example.org|
|     5|mcleanmatthew|johnrosales@examp...|
+------+-------------+--------------------+
only showing top 5 rows



In [20]:
(df_accounts
    .write
    .format("jdbc")
    .option("url", f"{url}/spark_prj_1_sample_dw")
    .option("driver", properties["driver"])
    .option("dbtable", "account_dim")
    .option("user", properties["user"])
    .option("password", properties["password"])
    .mode("append")
    .save()
)

In [21]:
df_acc_creation = (
    spark
    .read
    .format("jdbc")
    .option("url", f"{url}/spark_prj_1")
    .option("driver", properties["driver"])
    .option("dbtable", tables[1])
    .option("user", properties["user"])
    .option("password", properties["password"])
    .load()
)

In [22]:
df_acc_creation.show(5)

+---+-------------+----------+--------------------+-----------+-------+
| id|     username|  password|               email|createdDate|user_id|
+---+-------------+----------+--------------------+-----------+-------+
|  1|  lindarogers|8AfMY#Jb_e|rachel63@example.com| 2023-06-20|      1|
|  2|     joseph12|oGnA^KzC@5|jamesjesse@exampl...| 2022-02-01|      1|
|  3|   fhernandez|E$537_BmX+|fergusontonya@exa...| 2018-08-01|      2|
|  4|  paulpearson|$^k0NBg&70|   max00@example.org| 2018-11-09|      2|
|  5|mcleanmatthew|xhsKQ)n4^0|johnrosales@examp...| 2024-03-14|      2|
+---+-------------+----------+--------------------+-----------+-------+
only showing top 5 rows



In [23]:
df_acc_creation = (
    df_acc_creation
    .withColumnRenamed("id", "acc_id")
    .withColumnRenamed("createdDate", "created_date")
    .drop("username")
    .drop("password")
    .drop("email")
)

In [24]:
df_acc_creation.show(5)

+------+------------+-------+
|acc_id|created_date|user_id|
+------+------------+-------+
|     1|  2023-06-20|      1|
|     2|  2022-02-01|      1|
|     3|  2018-08-01|      2|
|     4|  2018-11-09|      2|
|     5|  2024-03-14|      2|
+------+------------+-------+
only showing top 5 rows



In [25]:
(df_acc_creation
    .write
    .format("jdbc")
    .option("url", f"{url}/spark_prj_1_sample_dw")
    .option("driver", properties["driver"])
    .option("dbtable", "account_creation_fact")
    .option("user", properties["user"])
    .option("password", properties["password"])
    .mode("append")
    .save()
)